In [1]:
import xarray as xr
target_vars = ["t_min_obs"]

dset = xr.open_zarr("/nesi/project/niwa03642/results/clidb_daily.zarr.zip")
dset = dset.sel(time='2018-02-22')# select 2018-02-22 due to its least missing data
dset = dset.dropna(dim='agent_no', subset=target_vars) # drop records 't_min_obs' are nan
dset = dset.drop_sel(agent_no=[5446, 3909, 4513, 1041]) # these sites has more than one record on one day

In [2]:
dset

<xarray.Dataset>
Dimensions:         (agent_no: 258, time: 1)
Coordinates:
  * agent_no        (agent_no) int64 1002 1056 1134 1196 ... 42704 42899 42975
    site_height     (agent_no) float64 dask.array<chunksize=(258,), meta=np.ndarray>
    site_latitude   (agent_no) float64 dask.array<chunksize=(258,), meta=np.ndarray>
    site_longitude  (agent_no) float64 dask.array<chunksize=(258,), meta=np.ndarray>
  * time            (time) datetime64[ns] 2018-02-22T09:00:00
Data variables:
    t_max_obs       (agent_no, time) float64 dask.array<chunksize=(58, 1), meta=np.ndarray>
    t_min_obs       (agent_no, time) float64 dask.array<chunksize=(58, 1), meta=np.ndarray>
    trange          (agent_no, time) float64 dask.array<chunksize=(58, 1), meta=np.ndarray>
    wind_speed_obs  (agent_no, time) float64 dask.array<chunksize=(58, 1), meta=np.ndarray>

In [3]:
X = dset.drop_vars(target_vars)
y = dset[list(target_vars)]

In [4]:
from src.smart_interpolation.models.anuspline import AnusplineModel
model = AnusplineModel(workdir="a_local_scratch_folder", use_norton=False, n_jobs=1)
model.fit(X, y)

/nesi/nobackup/niwa03642/wangc/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


AnusplineModel(workdir=PosixPath('a_local_scratch_folder'), use_norton=False, n_jobs=1, _y_train=<xarray.Dataset>
Dimensions:         (agent_no: 258, time: 1)
Coordinates:
  * agent_no        (agent_no) int64 1002 1056 1134 1196 ... 42704 42899 42975
    site_height     (agent_no) float64 213.0 79.0 204.0 ... 172.0 164.0 60.0
    site_latitude   (agent_no) float64 -34.43 -35.18 -35.42 ... -43.79 -46.12
    site_longitude  (agent_no) float64 172.7 173.9 173.8 ... 172.8 171.8 168.9
  * time            (time) datetime64[ns] 2018-02-22T09:00:00
Data variables:
    t_min_obs       (agent_no, time) float64 15.2 13.6 14.0 13.6 ... 8.7 5.2 8.1)

In [5]:
results = model.predict(X)

CalledProcessError: Command 'MODULEPATH=/opt/niwa/CS500_centos7_skl/modules/all module load ANUspline/43 && splina < a_local_scratch_folder/anuspline-7sltom9b/splina_pass1.ctl' returned non-zero exit status 127.